In [1]:
import numpy as np
import scipy.io
mat = scipy.io.loadmat('dataset_connected_NYC.mat') 

In [2]:
friendship_new = mat['friendship_new']
friendship_old = mat['friendship_old']#new_friendship_old
selected_checkins = mat['selected_checkins'] #new_checkins
selected_users_IDs = mat['selected_users_IDs'] #new_users_IDs

In [3]:
#do not run this if entire dataset is required
import random

qty = 10;

new_users_IDs = selected_users_IDs[:qty];

new_friendship_old = []
for i in range(np.size(friendship_old,0)):
    if ((friendship_old[i,0]<=qty) or (friendship_old[i,1]<=qty)):
        new_friendship_old.append(friendship_old[i,:])
        
new_friendship_old = np.array(new_friendship_old)


#make sure all qty user ids are accounted for atleast
#once in the friendship matrix
#found = np.zeros((qty,))
#for i in range(qty):
#    found[i] = i in new_friendship_old
#temp = np.where(found == 0)[0]; #ensure that all users are accounted for atleast once


for i in range(np.size(new_friendship_old,0)):
    if new_friendship_old[i,0]>qty:
        new_friendship_old[i,0]=random.randint(1,qty)
    elif new_friendship_old[i,1]>qty:
        new_friendship_old[i,1]=random.randint(1,qty)

new_checkins = []
for i in range(np.size(selected_checkins,0)):
    if selected_checkins[i,0]<=qty:
        new_checkins.append(selected_checkins[i,:])

#assigning original variables

selected_checkins = np.array(new_checkins)
friendship_old = np.array(new_friendship_old)
selected_users_IDs = new_users_IDs

#verify

print(selected_checkins)
print(friendship_old)
print(selected_users_IDs)


[[      1      46 3259566     120]
 [      1      68      62      44]
 [      1      13 5032552     230]
 [      4      37   15947     150]
 [      4      39 1748219      72]
 [      4      69 2260369     248]
 [      1      95     372      48]
 [      4     115   12743     243]
 [      4     120 2190699      61]
 [      4     134   67999     283]
 [      3     147   13320      44]
 [      1     160    7620     168]
 [      1      22   12743     243]
 [      9      57   18032     307]
 [      2      65 5323788      72]
 [      1      67    9828      72]
 [      2      70   12818     397]
 [      1      70     155      48]
 [      2      83 5793549      72]
 [      4     106   97131      70]
 [      1     110   10416       8]
 [      1     115    6344      80]
 [      4     133     555     307]
 [     10     139 1578583     380]
 [      1      47 5575535     163]
 [      1      95    1455      44]
 [      1     117 2475486      59]
 [      1     118 6193087     120]
 [      1     119   

In [4]:
# 1. rebuild node index
offset1 = max(selected_checkins[:,0]);
dumy, dumy, n = np.unique(selected_checkins[:,1],return_index=True,return_inverse=True, axis=0)


selected_checkins[:,1] = n+1#+offset1; #n is the indices of the unique values in selected_checkins[:,1]
offset2 = max(selected_checkins[:,1]);
dumy, dumy, n = np.unique(selected_checkins[:,2],return_index=True,return_inverse=True, axis=0)

selected_checkins[:,2] = n+1#+offset2;
offset3 = max(selected_checkins[:,2]);
dumy, dumy, n = np.unique(selected_checkins[:,3],return_index=True,return_inverse=True, axis=0)
selected_checkins[:,3] = n+1#+offset3;

num_node_total = max(map(max, selected_checkins)) #max of the entire matrix = 8117

In [5]:
#verify sequence
selected_checkins-=1
print(selected_checkins.shape)
print(min(selected_checkins[:,0]))
print(max(selected_checkins[:,0]))
print(min(selected_checkins[:,1]))
print(max(selected_checkins[:,1]))
print(min(selected_checkins[:,2]))
print(max(selected_checkins[:,2]))
print(min(selected_checkins[:,3]))
print(max(selected_checkins[:,3]))


(246, 4)
0
9
0
93
0
162
0
73


In [6]:
num_of_users = max(selected_checkins[:,0])
num_of_time = max(selected_checkins[:,1])
num_of_venues = max(selected_checkins[:,2])
num_of_categories = max(selected_checkins[:,3])

In [7]:
print(num_of_users)
print(num_of_time)
print(num_of_venues)
print(num_of_categories)

9
93
162
73


In [8]:
#separating training and testing data
train_size = int((80/100)*np.size(selected_checkins,0))
train_data = selected_checkins[:train_size]
nums_type = np.array([num_of_users+1, num_of_time+1, num_of_venues+1, num_of_categories+1])
test_data = selected_checkins[train_size:]
#data = {'train_data':train_data,'test_data':test_data,'num_types':num_types}

In [9]:
np.savez_compressed('lbsndata/train_data.npz', train_data = train_data, nums_type = nums_type)
np.savez_compressed('lbsndata/test_data.npz', test_data = test_data, nums_type = nums_type)


In [10]:
print(train_data.shape)

(196, 4)


In [11]:
#selected_checkins format -
#selected_checkins (4 columns): user_index, time (hour in a week), venue_index, venue_category_index
#feeding it as it is to DHNE

%run hypergraph_embedding.py --data_path lbsndata --save_path lbsnresult -s 100 100 100 100 -e 10

Using TensorFlow backend.


this is args
Namespace(alpha=1, batch_size=16, data_path='lbsndata', embedding_size=[100, 100, 100, 100], epochs_to_train=5, hidden_size=64, learning_rate=0.01, num_neg_samples=5, options=None, prefix_path='model', save_path='lbsnresult', seed=None)
train_data.edge [[  0  22 136  25]
 [  0  33   2   6]
 [  0   2 146  49]
 [  3  14  79  34]
 [  3  16 125  18]
 [  3  34 133  55]
 [  0  48   8   8]
 [  3  60  69  53]
 [  3  64 132  13]
 [  3  70 104  64]
 [  2  81  73   6]
 [  0  86  47  42]
 [  0   8  69  53]
 [  8  25  87  69]
 [  1  31 149  18]
 [  0  32  59  18]
 [  1  35  70  73]
 [  0  35   4   8]
 [  1  40 152  18]
 [  3  51 108  17]
 [  0  55  61   0]
 [  0  60  41  19]
 [  3  69  10  69]
 [  9  74 124  72]
 [  0  23 150  39]
 [  0  48  21   6]
 [  0  61 134  12]
 [  0  62 156  25]
 [  0  63  60   2]
 [  0  65 113  14]
 [  0  66 134  12]
 [  0  75   3   6]
 [  0  18  99  58]
 [  0  22   1   6]
 [  0  26  75  58]
 [  0  45   9   5]
 [  3  46   0   2]
 [  0  46   2   6]
 [  0  54  7

Epoch 1/5
13/13 [==============================] - 3s 217ms/step - loss: 0.8486 - decode_0_loss: 0.0104 - decode_1_loss: 0.0019 - decode_2_loss: 0.0023 - decode_3_loss: 0.0032 - classify_layer_loss: 0.8308 - decode_0_mean_squared_error: 0.4192 - decode_1_mean_squared_error: 0.3734 - decode_2_mean_squared_error: 0.3198 - decode_3_mean_squared_error: 0.3709 - classify_layer_acc: 0.7532 - val_loss: 0.4281 - val_decode_0_loss: 0.0013 - val_decode_1_loss: 5.3789e-04 - val_decode_2_loss: 4.4519e-04 - val_decode_3_loss: 7.3765e-04 - val_classify_layer_loss: 0.4250 - val_decode_0_mean_squared_error: 0.4551 - val_decode_1_mean_squared_error: 0.4213 - val_decode_2_mean_squared_error: 0.3824 - val_decode_3_mean_squared_error: 0.4454 - val_classify_layer_acc: 0.8333
Epoch 2/5
13/13 [==============================] - 0s 11ms/step - loss: 0.4293 - decode_0_loss: 9.5996e-04 - decode_1_loss: 4.6193e-04 - decode_2_loss: 8.3057e-04 - decode_3_loss: 7.8394e-04 - classify_layer_loss: 0.4263 - decode_0_mea

In [12]:
#embeddings are stored in lbsnresult/model_(no.ofemneddings)/embeddings.npy
#corresponding model stored in lbsnresult/model_(no.ofemneddings)/model.h5

embed = np.load('lbsnresult/model_100/embeddings.npy')
print(embed.shape)
print('embed[0].shape',embed[0].shape)
print('embed[1].shape',embed[1].shape)
print('embed[2].shape',embed[2].shape)
print('embed[3].shape',embed[3].shape)

(4,)
embed[0].shape (10, 100)
embed[1].shape (94, 100)
embed[2].shape (163, 100)
embed[3].shape (74, 100)


In [45]:
#location prediction on the learnt embeddings
#selected_checkins (4 columns): user_index, time (hour in a week), venue_index, venue_category_index
import time as tm
from scipy.spatial import distance

count = 0

start = tm.time()
for d in range(test_data.shape[0]):
    test = test_data[d]
    user = test[0]
    time = test[1]
    dist = []
    
    for i in range(min(test_data[:,2]),max(test_data[:,2])+1):
        summ = (1 - distance.cosine(embed[0][user],embed[2][i]))+(1 - distance.cosine(embed[1][time],embed[2][i]))
        dist.append((abs(summ),i))
        
    dist = sorted(dist, key=lambda student: student[0])
    if test[2] in (np.array(dist[0:20])[:,1]):
        count+=1
        
ending = tm.time()

print("this is count ",count)
print("this is count fraction ",count/test_data.shape[0])
print("time lapsed ",ending-start)

this is count  3
this is count fraction  0.06
time lapsed  0.8240852355957031
